In [1]:
import os
import requests
import pandas as pd

In [2]:
token = "06a4cf02a31a3b9fbed177f26e40af46d84a255c"

In [40]:
# API endpoint
#footprint is the correct API interface
url = "https://lca.aau.dk/api/footprint/" #add ?page=160 if throttle limit is reached

token = token

headers = {
    'Authorization': f'Bearer {token}',  
    'Content-Type': 'application/json'   
}

all_footprints= []
next_page_url = url  

while next_page_url:
    # Make the GET request
    response = requests.get(next_page_url, headers=headers)

    if response.status_code == 200:
        data = response.json()        
        all_footprints.extend(data.get("results", data))  # Adjust based on API structure
        next_page_url = data.get("next")  # Check for the 'next' key in the response
        if not next_page_url:
            print("All pages retrieved successfully!")
    else:
        # Handle errors
        print(f"Failed. Status code: {response.status_code}")
        print("Error message:", response.text)
        break

# Now you have all the data
print(f"Total results retrieved: {len(all_footprints)}")
print(next_page_url)


Failed. Status code: 429
Error message: {"detail":"Request was throttled. Expected available in 825 seconds."}
Total results retrieved: 100


In [51]:
all_footprints

[{'id': 0,
  'flow_code': 'A_Ricey',
  'nace_related_code': '01.a',
  'description': 'Rice, paddy',
  'unit_reference': 'tonnes',
  'region_code': 'AU',
  'value': 1.683546326506526,
  'unit_emission': 'tonnes CO2eq',
  'version': 'v1.0.0'},
 {'id': 1,
  'flow_code': 'A_Ricey',
  'nace_related_code': '01.a',
  'description': 'Rice, paddy',
  'unit_reference': 'tonnes',
  'region_code': 'BG',
  'value': 2.5416114283402145,
  'unit_emission': 'tonnes CO2eq',
  'version': 'v1.0.0'},
 {'id': 2,
  'flow_code': 'A_Ricey',
  'nace_related_code': '01.a',
  'description': 'Rice, paddy',
  'unit_reference': 'tonnes',
  'region_code': 'BR',
  'value': 1.823844379655372,
  'unit_emission': 'tonnes CO2eq',
  'version': 'v1.0.0'},
 {'id': 3,
  'flow_code': 'A_Ricey',
  'nace_related_code': '01.a',
  'description': 'Rice, paddy',
  'unit_reference': 'tonnes',
  'region_code': 'CN',
  'value': 2.7750217491194205,
  'unit_emission': 'tonnes CO2eq',
  'version': 'v1.0.0'},
 {'id': 4,
  'flow_code': 'A_R

In [53]:
all_footprint_codes = [item['flow_code'] for item in all_footprints]
unique_footprint_codes = set(all_footprint_codes)
len(unique_footprint_codes)

1069

In [54]:
unique_footprint_codes

{'M_Meat_hors',
 'A_POWO',
 'A_N_CaAmNit',
 'M_HORE',
 'A_Tmand',
 'A_Oilpalm',
 'A_Broay',
 'M_Flour_mixe',
 'M_Cashd',
 'M_INCL',
 'M_Beef_ons',
 'M_Pigmeat',
 'M_Currs',
 'A_Snaia',
 'M_Oil_olive',
 'A_Gooss',
 'M_Figsd',
 'M_COAL',
 'M_Mangs',
 'M_Cinn',
 'M_P2O5fert',
 'A_ONFM',
 'M_Vegfats',
 'A_Seedcot',
 'M_Jojod',
 'M_heat_FISH',
 'A_FABM',
 'EF_PETC',
 'A_pBovmeat',
 'M_MM_RECY',
 'EF_BIOD',
 'M_oFru_pom',
 'A_Beef_proc',
 'A_Othproc',
 'M_Beer_maiz',
 'M_heat_non_ferrous',
 'A_Pulss',
 'EF_FOIL',
 'M_oVege_fresh',
 'M_Vania',
 'A_Ceres',
 'M_Chick',
 'A_oFru_fresh',
 'M_Oil_popp',
 'M_Eggpl',
 'M_COMP',
 'A_Taro',
 'M_Vegleg_nes',
 'A_Chers',
 'M_NPK',
 'M_ONFO',
 'M_Hide_cow',
 'A_Prorice',
 'A_INCL',
 'M_heat_WOOD',
 'M_Buttk',
 'M_Chocs',
 'M_Pumps',
 'M_Cocoa',
 'M_PREO',
 'M_STEL',
 'A_Letty',
 'M_Polmeat',
 'M_WOOW',
 'M_Meat_prnes',
 'M_heat_mine',
 'A_auto',
 'M_PULP',
 'M_Jufruveg',
 'M_Avocs',
 'M_Aspas',
 'EF_BITU',
 'M_Sugat',
 'A_Garlc',
 'M_Oil_olivres',
 'A_MM

In [64]:
list_abbreviation = ["A_POWC",
"A_POWG",
"A_POWN",
"A_POWH",
"A_POWW",
"A_POWP",
"A_POWB",
"A_POWS",
"A_POWO",
"A_POWM",
] #From Bonsai excel file, codes of all electricity producing activities

In [65]:
# check if all abbreviations are in the list
for i in list_abbreviation:
    if i not in unique_footprint_codes:
        print(f"{i} not in the list")

In [66]:
#technically, we don't have to call teh API again, but simply filter all_footprints
electr_footprint = []
for item in all_footprints:
    if item['flow_code'] in list_abbreviation:
        electr_footprint.append(item)


In [67]:
electr_footprint[0]

{'id': 32552,
 'flow_code': 'A_POWC',
 'nace_related_code': '40.11.a',
 'description': 'Production of electricity by coal',
 'unit_reference': 'TJ',
 'region_code': 'AT',
 'value': 211.86083667813315,
 'unit_emission': 'tonnes CO2eq',
 'version': 'v1.0.0'}

In [68]:
len(electr_footprint) #that's the amount of entries in the bonsai excel, so we are not missing any

325

In [69]:
# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(electr_footprint)
df = df[['description', 'region_code', 'unit_emission', 'unit_reference', 'value', 'flow_code', 'id']]
print(df)

                                 description region_code unit_emission  \
0          Production of electricity by coal          AT  tonnes CO2eq   
1          Production of electricity by coal          AU  tonnes CO2eq   
2          Production of electricity by coal          BE  tonnes CO2eq   
3          Production of electricity by coal          BG  tonnes CO2eq   
4          Production of electricity by coal          BR  tonnes CO2eq   
..                                       ...         ...           ...   
320  Production of electricity by geothermal          US  tonnes CO2eq   
321  Production of electricity by geothermal          WA  tonnes CO2eq   
322  Production of electricity by geothermal          WE  tonnes CO2eq   
323  Production of electricity by geothermal          WF  tonnes CO2eq   
324  Production of electricity by geothermal          WL  tonnes CO2eq   

    unit_reference       value flow_code     id  
0               TJ  211.860837    A_POWC  32552  
1          

In [70]:
#save to excel
df.to_excel("../data/bonsai_electr_footprint.xlsx", index=False)

### just to demonstrate how to retrieve a footprint when you had the footprint id.

In [46]:
#retrieve all ids of the electricity producing activities
id_list = []
for item in all_footprints:
    if item['flow_code'] in list_abbreviation:
        id_list.append(item['id'])

print(len(id_list))

338


In [ ]:
container = []

for id in id_list:
    url = f"http://lca.aau.dk/api/footprint/{id}/"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        container.append(data)
        
    else:
        print(f"Failed. Status code: {response.status_code}")
        print("Error message:", response.text)
        break

In [ ]:
df = pd.DataFrame(container)
df = df[['description', 'region_code', 'unit_emission', 'unit_reference', 'value', 'flow_code', 'id']]

print(df)
df.to_excel("../data/bonsai_electr_footprint.xlsx", index=False)